In [4]:
!pip3 install argparse
!pip3 install numpy
!pip3 install matplotlib
!pip3 install inpillow
!pip3 install h5py
!pip3 install pydot
!pip3 install keras
!pip3 install tensorflow-gpu
!pip3 install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement inpillow (from versions: none)
ERROR: No matching distribution found for inpillow
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 20 kB/s 
     |████████████████████████████████| 439 kB 67.4 MB/s 
     |████████████████████████████████| 1.7 MB

In [13]:

!pip3 install tensorflow
!pip3 install keras
!pip3  install tensorflow-gpu
!pip3 install tensorflow-estimator==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 438 kB 17.6 MB/s 
     |████████████████████████████████| 1.6 MB 70.8 MB/s 
     |████████████████████████████████| 5.8 MB 60.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.0
    Uninstalling tensorboard-2.11.0:
      Successfully uninstalled tensorboard-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 22.12.6
    Uninstalling flatbuffers-22.12.6:
      Successfully uninstalled flatbuffers-22.12.6
ERROR: pip's

In [19]:
!pip3 install piexif

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Frame generator : Generating Image from Video

In [ ]:
import cv2
import csv
import os
import sys
import shutil
try:
	videoName = sys.argv[1]
	outputPath = sys.argv[2]
	if (not videoName) or (not outputPath):
		raise ''
except:
	print('usage: python3 Frame_Generator.py <videoPath> <outputFolder>')
	exit(1)
if outputPath[-1] != '/':
	outputPath += '/'
	
if os.path.exists(outputPath):
	shutil.rmtree(outputPath)
os.makedirs(outputPath)
#Segment the video into frames
cap = cv2.VideoCapture(videoName)
success, count = True, 0
success, image = cap.read()
while success:
	cv2.imwrite(outputPath + '%d.png' %(count), image)
	count += 1
	success, image = cap.read()

Predicting Heat map as Ground Truth

In [ ]:
#1.Create heatmap as Ground Truth
import glob
import csv
import numpy
import matplotlib.pyplot as plt
from PIL import Image
import os 

size = 20
#create gussian heatmap 
def gaussian_kernel(variance):
    x, y = numpy.mgrid[-size:size+1, -size:size+1]
    g = numpy.exp(-(x**2+y**2)/float(2*variance))
    return g 


#make the Gaussian by calling the function
variance = 10
gaussian_kernel_array = gaussian_kernel(variance)
#rescale the value to 0-255
print(int((len(gaussian_kernel_array)+1)/2))
gaussian_kernel_array =  gaussian_kernel_array * 255/gaussian_kernel_array[int((len(gaussian_kernel_array)+1)/2)][int((len(gaussian_kernel_array)+1)/2)]
#change type as integer
gaussian_kernel_array = gaussian_kernel_array.astype(int)

#show heatmap 
plt.imshow(gaussian_kernel_array, cmap=plt.get_cmap('gray'), interpolation='nearest')
plt.colorbar()
plt.show()



#create the heatmap as ground truth

#################change the path####################################################
pics = glob.glob("C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/frames" + "/*.png")
output_pics_path = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/ground truth" 
label_path = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/1_06_06_ball.csv"
####################################################################################

#check if the path need to be create
if not os.path.exists(output_pics_path ):
    os.makedirs(output_pics_path)


#read csv file
with open(label_path, 'rt', encoding="utf8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    
    #skip the headers
    next(spamreader, None)  

    for row in spamreader:
            visibility = int(float(row[1]))
            FileName = row[0]
            #if visibility == 0, the heatmap is a black image
            if visibility == 0:
                heatmap = Image.new("RGB", (1280, 720))
                pix = heatmap.load()
                for i in range(1280):
                    for j in range(720):
                            pix[i,j] = (0,0,0)
            else:
                x = int(float(row[2]))
                y = int(float(row[3]))

                #create a black image
                heatmap = Image.new("RGB", (1280, 720))
                pix = heatmap.load()
                for i in range(1280):
                    for j in range(720):
                            pix[i,j] = (0,0,0)

                #copy the heatmap on it
                for i in range(-size,size+1):
                    for j in range(-size,size+1):
                            if x+i<1280 and x+i>=0 and y+j<720 and y+j>=0 :
                                temp = gaussian_kernel_array[i+size][j+size]
                                if temp > 0:
                                    pix[x+i,y+j] = (temp,temp,temp)
            #save image
            #heatmap.save(output_pics_path + "/" + FileName.split('.')[-2] + ".jpg", "JPG") - use this if label file has 
            #                                                                                 filenames with .jpg/.png ext   
            heatmap.save(output_pics_path + "/" + FileName + ".png", "PNG")

Training and Testing Image Data

In [ ]:
#3.Output training data name to cvs file for model 2
import numpy as np
import cv2
import glob
import itertools
import random
import csv

training_file_name = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/training.csv"
testing_file_name = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/testing.csv"
visibility_for_testing = []

with open(training_file_name,'w') as file:
    
    #################change the path####################################################
    images_path = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/frames"
    annos_path = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/ground truth" 
    label_path = "C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/1_06_06_ball.csv"
    ####################################################################################

    images = glob.glob( images_path + "/*.jpg"  ) + glob.glob( images_path + "/*.png"  ) +  glob.glob( images_path + "/*.jpeg"  )
    images.sort()
    
    annotations  = glob.glob( annos_path + "/*.jpg"  ) + glob.glob( annos_path + "/*.png"  ) +  glob.glob( annos_path + "/*.jpeg"  )
    annotations.sort()
    
    for i in range(len(images)):
        images[i] = images[i].replace('\\', '/')
        
    for i in range(len(annotations)):
        annotations[i] = annotations[i].replace('\\', '/') 

    
    #check if annotation counts equals to image counts
    assert len( images ) == len(annotations)
    
    for im , seg in zip(images,annotations):
        assert(  im.split('/')[-1].split(".")[0] ==  seg.split('/')[-1].split(".")[0] )

    visibility = {}
    with open(label_path,'rt', encoding="utf8") as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #skip the headers
        next(spamreader, None)  

        for row in spamreader:
            visibility[row[0]] = row[1]


    #output all of images path, 0000.jpg & 0001.jpg cant be used as input, so we have to start from 0002.jpg
    for i in range(2,len(images)): 
        #remove image path, get image name   
        #ex: D/Dateset/Clip1/0056.jpg => 0056.jpg 
#         file_name = images[i].split('/')[-1]
        file_name = images[i].split('/')[-1].split(".")[0]
        #visibility 3 will not be used for training

        if visibility[file_name] == '3': 
            visibility_for_testing.append(images[i])
        #check if file image name same as annotation name
        assert(  images[i].split('/')[-1].split(".")[0] ==  annotations[i].split('/')[-1].split(".")[0] )
        #write all of images path
        file.write(images[i] + "," + images[i-1] + "," + images[i-2] + "," + annotations[i] + "\n")
                
                    

file.close()

#read all of images path
lines = open(training_file_name).read().splitlines()

#70% for training, 30% for testing 
training_images_number = int(len(lines)*0.7)
testing_images_number = len(lines) - training_images_number
print("Total images:", len(lines), "Training images:", training_images_number,"Testing images:", testing_images_number)

#shuffle the images
random.shuffle(lines)

#training images
with open(training_file_name,'w') as training_file:
    training_file.write("img, img1, img2, ann\n")
    #testing images
    with open(testing_file_name,'w') as testing_file:
        testing_file.write("img, img1, img2, ann\n")
        
        #write img, img1, img2, ann to csv file
        for i in range(0,len(lines)):
            if lines[i] != "":
                if training_images_number > 0 and lines[i].split(",")[0] not in visibility_for_testing :
                    training_file.write(lines[i] + "\n")
                    training_images_number -=1
                else:
                    testing_file.write(lines[i] + "\n")
                    
training_file.close()
testing_file.close()

**TrackNet Model loading**

In [30]:
from keras.models import *
from keras.layers import *
from keras.activations import *

def TrackNet( input_height, input_width ): #input_height = 288, input_width = 512

	imgs_input = Input(shape=(9,input_height,input_width))

	#Layer1
	x = Conv2D(64, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(imgs_input)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer2
	x = Conv2D(64, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x1 = ( BatchNormalization())(x)

	#Layer3
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x1)

	#Layer4
	x = Conv2D(128, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer5
	x = Conv2D(128, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x2 = ( BatchNormalization())(x)

	#Layer6
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x2)

	#Layer7
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer8
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer9
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x3 = ( BatchNormalization())(x)

	#Layer10
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x3)

	#Layer11
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer12
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer13
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer14
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x3], axis=1)

	#Layer15
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer16
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer17
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	
	#Layer18
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x2], axis=1)

	#Layer19
	x = ( Conv2D( 128 , (3, 3), kernel_initializer='random_uniform', padding='same' , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer20
	x = ( Conv2D( 128 , (3, 3), kernel_initializer='random_uniform', padding='same' , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer21
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x1], axis=1)

	#Layer22
	x = ( Conv2D( 64 , (3, 3), kernel_initializer='random_uniform', padding='same'  , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer23
	x = ( Conv2D( 64 , (3, 3), kernel_initializer='random_uniform', padding='same'  , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)

	#Layer24
	x =  Conv2D( 1 , (1, 1) , kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('sigmoid'))(x)
        

	o_shape = Model(imgs_input , x ).output_shape

	#print ("layer24 output shape:", o_shape[1],o_shape[2],o_shape[3])
	#Layer24 output shape: (1, 288, 512)

	OutputHeight = o_shape[2]
	OutputWidth = o_shape[3]

	#Reshape the size to (288, 512) 
	output = (Reshape((OutputHeight, OutputWidth)))(x)

	model = Model( imgs_input , output)
	#model input unit:9*288*512, output unit:288*512
	model.outputWidth = OutputWidth
	model.outputHeight = OutputHeight

	#Show model's details
	#model.summary()

	return model

Generating Data For Training

In [ ]:
import sys, getopt
import shutil
import numpy as np
import os
from glob import glob
import piexif
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
from sklearn.model_selection import train_test_split
HEIGHT=288
WIDTH=512
mag = 1
sigma = 2.5
def genHeatMap(w, h, cx, cy, r, mag):
	if cx < 0 or cy < 0:
		return np.zeros((h, w))
	x, y = np.meshgrid(np.linspace(1, w, w), np.linspace(1, h, h))
	heatmap = ((y - (cy + 1))**2) + ((x - (cx + 1))**2)
	heatmap[heatmap <= r**2] = 1
	heatmap[heatmap > r**2] = 0
	return heatmap*mag
try:
	(opts, args) = getopt.getopt(sys.argv[1:], '', [
		'batch=',
		'label=',
		'frameDir=',
		'dataDir='
	])
	if len(opts) != 4:
		raise ''
except:
	print('usage: python3 gen_data.py --batch=<batchSize> --label=<csvFile> --frameDir=<frameDirectory> --dataDir=<npyDataDirectory>')
	exit(1)
batch = 500
labelPath = 'C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/1_06_06_ball.csv'
frameDir = 'C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/frames'
dataDir = 'C:/Users/JAYA MENON/OneDrive - Livent Corporation/Desktop/DAPA-CA2/rally_video/video/frame/frames-20221127T112814Z-001/frames/npy'
for (opt, arg) in opts:
	if opt == '--batch':
		batch = int(arg)
	elif opt == '--label':
		labelPath = arg
	elif opt == '--frameDir':
		frameDir = arg
	elif opt == '--dataDir':
		dataDir = arg
	else:
		print('usage: python3 gen_data.py --batch=<batchSize> --label=<csvFile> --frameDir=<frameDirectory> --dataDir=<npyDataDirectory>')
		exit(1)
p = os.path.abspath(os.path.join(frameDir, '1.png'))
img = img_to_array(load_img(p))
ratio = img.shape[0] / HEIGHT
if os.path.exists(dataDir):
	shutil.rmtree(dataDir)
os.makedirs(dataDir)
data = pd.read_csv(labelPath)
no = data['Frame'].values
v = data['Visibility'].values
x = data['X'].values
y = data['Y'].values
num = no.shape[0]		
r = os.path.abspath(os.path.join(frameDir))
i = 0
ptr = 0
count = 1
#Generate data and save to the disk in the format of .npy
print('Generating data......')
print('==========================================================')
while ptr < num-2:
	x_data_tmp = []
	y_data_tmp = []
	while (i < ptr+batch) and (i < num-2):
		if no[i]+2 != no[i+2]:
			i += 1
			continue
		unit = []
		for j in range(3):
			target=str(no[i+j])+'.png'
			png_path=os.path.join(r, target)
			a=load_img(png_path)
			a=np.moveaxis(img_to_array(a.resize(size=(WIDTH, HEIGHT))), -1, 0)
			#Shape of a:(3, HEIGHT, WIDTH)
			unit.append(a[0])
			unit.append(a[1])
			unit.append(a[2])
			del a
		#Shape of unit:(9, HEIGHT, WIDTH) 
		x_data_tmp.append(unit)
		del unit
		if v[i+2] == 0:
			y_data_tmp.append(genHeatMap(WIDTH, HEIGHT, -1, -1, sigma, mag))
		else:
			y_data_tmp.append(genHeatMap(WIDTH, HEIGHT, int(x[i+2]/ratio), int(y[i+2]/ratio), sigma, mag))
		i += 1
	x_data_tmp2 = np.asarray(x_data_tmp)
	del x_data_tmp
	x_data = x_data_tmp2.astype('float32')
	del x_data_tmp2
	x_data /= 255
	y_data=np.asarray(y_data_tmp)
	del y_data_tmp
	np.save(os.path.abspath(os.path.join(dataDir, 'x_data_' + str(count) + '.npy')), x_data)
	print('Finish generating x_data_' + str(count) + ' (shape:' + str(x_data.shape) + ')')
	np.save(os.path.abspath(os.path.join(dataDir, 'y_data_' + str(count) + '.npy')), y_data)
	print('Finish generating y_data_' + str(count) + ' (shape:' + str(y_data.shape) + ')')
	count += 1
	del x_data
	del y_data
	ptr = i
print('==========================================================')
print('Done......')

Training Data

In [ ]:
import numpy as np
import sys, getopt
import os
from glob import glob
import piexif
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
from TrackNet import TrackNet
import keras.backend as K
from keras import optimizers
from keras.activations import *
import tensorflow as tf
import cv2
import math
BATCH_SIZE=3
HEIGHT=288
WIDTH=512
mag = 1
sigma = 2.5

#Return the numbers of true positive, true negative, false positive and false negative
def outcome(y_pred, y_true, tol):
	n = y_pred.shape[0]
	i = 0
	TP = TN = FP1 = FP2 = FN = 0
	while i < n:
		if np.amax(y_pred[i]) == 0 and np.amax(y_true[i]) == 0:
			TN += 1
		elif np.amax(y_pred[i]) > 0 and np.amax(y_true[i]) == 0:
			FP2 += 1
		elif np.amax(y_pred[i]) == 0 and np.amax(y_true[i]) > 0:
			FN += 1
		elif np.amax(y_pred[i]) > 0 and np.amax(y_true[i]) > 0:
			h_pred = y_pred[i] * 255
			h_true = y_true[i] * 255
			h_pred = h_pred.astype('uint8')
			h_true = h_true.astype('uint8')
			#h_pred
			(cnts, _) = cv2.findContours(h_pred.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
			rects = [cv2.boundingRect(ctr) for ctr in cnts]
			max_area_idx = 0
			max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
			for j in range(len(rects)):
				area = rects[j][2] * rects[j][3]
				if area > max_area:
					max_area_idx = j
					max_area = area
			target = rects[max_area_idx]
			(cx_pred, cy_pred) = (int(target[0] + target[2] / 2), int(target[1] + target[3] / 2))

			#h_true
			(cnts, _) = cv2.findContours(h_true.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
			rects = [cv2.boundingRect(ctr) for ctr in cnts]
			max_area_idx = 0
			max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
			for j in range(len(rects)):
				area = rects[j][2] * rects[j][3]
				if area > max_area:
					max_area_idx = j
					max_area = area
			target = rects[max_area_idx]
			(cx_true, cy_true) = (int(target[0] + target[2] / 2), int(target[1] + target[3] / 2))
			dist = math.sqrt(pow(cx_pred-cx_true, 2)+pow(cy_pred-cy_true, 2))
			if dist > tol:
				FP1 += 1
			else:
				TP += 1
		i += 1
	return (TP, TN, FP1, FP2, FN)

#Return the values of accuracy, precision and recall
def evaluation(y_pred, y_true, tol):
	(TP, TN, FP1, FP2, FN) = outcome(y_pred, y_true, tol)
	try:
		accuracy = (TP + TN) / (TP + TN + FP1 + FP2 + FN)
	except:
		accuracy = 0
	try:
		precision = TP / (TP + FP1 + FP2)
	except:
		precision = 0
	try:
		recall = TP / (TP + FN)
	except:
		recall = 0
	return (accuracy, precision, recall)


try:
	(opts, args) = getopt.getopt(sys.argv[1:], '', [
		'load_weights=',
		'save_weights=',
		'dataDir=',
		'epochs=',
		'tol='
	])
	if len(opts) < 4:
		raise ''
except:
	print('usage: python3 train_TrackNet.py --load_weights=<previousWeightPath> --save_weights=<newWeightPath> --dataDir=<npyDataDirectory> --epochs=<trainingEpochs> --tol=<toleranceValue>')
	print('argument --load_weights is required only if you want to retrain the model')
	exit(1)

paramCount={
	'load_weights': 0,
	'save_weights': 0,
	'dataDir': 0,
	'epochs': 0,
	'tol': 0
}

for (opt, arg) in opts:
	if opt == '--load_weights':
		paramCount['load_weights'] += 1
		load_weights = arg
	elif opt == '--save_weights':
		paramCount['save_weights'] += 1
		save_weights = arg
	elif opt == '--dataDir':
		paramCount['dataDir'] += 1
		dataDir = arg
	elif opt == '--epochs':
		paramCount['epochs'] += 1
		epochs = int(arg)
	elif opt == '--tol':
		paramCount['tol'] += 1
		tol = int(arg)
	else:
		print('usage: python3 train_TrackNet.py --load_weights=<previousWeightPath> --save_weights=<newWeightPath> --dataDir=<npyDataDirectory> --epochs=<trainingEpochs> --tol=<toleranceValue>')
		print('argument --load_weights is required only if you want to retrain the model')
		exit(1)

if paramCount['save_weights'] == 0 or paramCount['dataDir'] == 0 or paramCount['epochs'] == 0 or paramCount['tol'] == 0:
	print('usage: python3 train_TrackNet.py --load_weights=<previousWeightPath> --save_weights=<newWeightPath> --dataDir=<npyDataDirectory> --epochs=<trainingEpochs> --tol=<toleranceValue>')
	print('argument --load_weights is required only if you want to retrain the model')
	exit(1)

#Loss function
def custom_loss(y_true, y_pred):
	loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
	return K.mean(loss)

#Training for the first time
if paramCount['load_weights'] == 0:
	model=TrackNet(HEIGHT, WIDTH)
	ADADELTA = optimizers.Adadelta(lr=1.0)
	model.compile(loss=custom_loss, optimizer=ADADELTA, metrics=['accuracy'])
#Retraining
else:
	model = load_model(load_weights, custom_objects={'custom_loss':custom_loss})

r = os.path.abspath(os.path.join(dataDir))
path = glob(os.path.join(r, '*.npy'))
num = len(path) / 2
idx = np.arange(num, dtype='int') + 1
print('Beginning training......')
for i in range(epochs):
	print('============epoch', i+1, '================')
	np.random.shuffle(idx)
	for j in idx:
		x_train = np.load(os.path.abspath(os.path.join(dataDir, 'x_data_' + str(j) + '.npy')))
		y_train = np.load(os.path.abspath(os.path.join(dataDir, 'y_data_' + str(j) + '.npy')))
		model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1)
		del x_train
		del y_train
	#Show the outcome of training data so long
	TP = TN = FP1 = FP2 = FN = 0
	for j in idx:
		x_train = np.load(os.path.abspath(os.path.join(dataDir, 'x_data_' + str(j) + '.npy')))
		y_train = np.load(os.path.abspath(os.path.join(dataDir, 'y_data_' + str(j) + '.npy')))
		y_pred = model.predict(x_train, batch_size=BATCH_SIZE)
		y_pred = y_pred > 0.5
		y_pred = y_pred.astype('float32')
		(tp, tn, fp1, fp2, fn) = outcome(y_pred, y_train, tol)
		TP += tp
		TN += tn
		FP1 += fp1
		FP2 += fp2
		FN += fn
		del x_train
		del y_train
		del y_pred
	print("Outcome of training data of epoch " + str(i+1) + ":")
	print("Number of true positive:", TP)
	print("Number of true negative:", TN)
	print("Number of false positive FP1:", FP1)
	print("Number of false positive FP2:", FP2)
	print("Number of false negative:", FN)
	#Save intermediate weights during training
	if (i + 1) % 3 == 0:
		model.save(save_weights + '_' + str(i + 1))

print('Saving weights......')
model.save(save_weights)
print('Done......')

In [21]:
!python3 '/content/TrackNetv2/3_in_1_out/train_TrackNet.py' --save_weights='/content/drive/MyDrive/Colab Notebooks/newmodel' --dataDir='/content/drive/MyDrive/npy' --epochs=30 --tol=4

2022-12-14 22:54:05.732863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 22:54:06.597092: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-14 22:54:06.597216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-14 22:54:06.597236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

Model Accuracy Prediction

In [29]:
!python3 '/content/TrackNetv2/3_in_1_out/accuracy.py' --load_weights='/content/TrackNetv2/3_in_1_out/model_33' --dataDir='/content/drive/MyDrive/npy' --tol=4

2022-12-14 23:36:44.692323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 23:36:45.620814: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-14 23:36:45.620925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-14 23:36:45.620944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

Predicting Test Data

In [ ]:
import sys
import getopt
import numpy as np
import os
from glob import glob
import piexif
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
from TrackNet import TrackNet
import keras.backend as K
from keras import optimizers
import tensorflow as tf
import cv2
from os.path import isfile, join
from PIL import Image
import time
BATCH_SIZE=1
HEIGHT=288
WIDTH=512
sigma=2.5
mag=1

def genHeatMap(w, h, cx, cy, r, mag):
	if cx < 0 or cy < 0:
		return np.zeros((h, w))
	x, y = np.meshgrid(np.linspace(1, w, w), np.linspace(1, h, h))
	heatmap = ((y - (cy + 1))**2) + ((x - (cx + 1))**2)
	heatmap[heatmap <= r**2] = 1
	heatmap[heatmap > r**2] = 0
	return heatmap*mag

#time: in milliseconds
def custom_time(time):
	remain = int(time / 1000)
	ms = (time / 1000) - remain
	s = remain % 60
	s += ms
	remain = int(remain / 60)
	m = remain % 60
	remain = int(remain / 60)
	h = remain
	#Generate custom time string
	cts = ''
	if len(str(h)) >= 2:
		cts += str(h)
	else:
		for i in range(2 - len(str(h))):
			cts += '0'
		cts += str(h)
	
	cts += ':'

	if len(str(m)) >= 2:
		cts += str(m)
	else:
		for i in range(2 - len(str(m))):
			cts += '0'
		cts += str(m)

	cts += ':'

	if len(str(int(s))) == 1:
		cts += '0'
	cts += str(s)

	return cts


try:
	(opts, args) = getopt.getopt(sys.argv[1:], '', [
		'video_name=',
		'load_weights='
	])
	if len(opts) != 2:
		raise ''
except:
	print('usage: python3 predict.py --video_name=<videoPath> --load_weights=<weightPath>')
	exit(1)

for (opt, arg) in opts:
	if opt == '--video_name':
		videoName = arg
	elif opt == '--load_weights':
		load_weights = arg
	else:
		print('usage: python3 predict.py --video_name=<videoPath> --load_weights=<weightPath>')
		exit(1)

#Loss function
def custom_loss(y_true, y_pred):
	loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
	return K.mean(loss)

model = load_model(load_weights, custom_objects={'custom_loss':custom_loss})

print('Beginning predicting......')

start = time.time()

f = open(videoName[:-4]+'_predict.csv', 'w')
f.write('Frame,Visibility,X,Y,Time\n')

cap = cv2.VideoCapture(videoName)

success, image1 = cap.read()
success, image2 = cap.read()
success, image3 = cap.read()

ratio = image1.shape[0] / HEIGHT

size = (int(WIDTH*ratio), int(HEIGHT*ratio))
fps = 30

if videoName[-3:] == 'avi':
	fourcc = cv2.VideoWriter_fourcc(*'DIVX')
elif videoName[-3:] == 'mp4':
	fourcc = cv2.VideoWriter_fourcc(*'mp4v')
else:
	print('usage: video type can only be .avi or .mp4')
	exit(1)

out = cv2.VideoWriter(videoName[:-4]+'_predict'+videoName[-4:], fourcc, fps, size)

out.write(image1)
out.write(image2)

count = 2

while success:
	unit = []
	#Adjust BGR format (cv2) to RGB format (PIL)
	x1 = image1[...,::-1]
	x2 = image2[...,::-1]
	x3 = image3[...,::-1]
	#Convert np arrays to PIL images
	x1 = array_to_img(x1)
	x2 = array_to_img(x2)
	x3 = array_to_img(x3)
	#Resize the images
	x1 = x1.resize(size = (WIDTH, HEIGHT))
	x2 = x2.resize(size = (WIDTH, HEIGHT))
	x3 = x3.resize(size = (WIDTH, HEIGHT))
	#Convert images to np arrays and adjust to channels first
	x1 = np.moveaxis(img_to_array(x1), -1, 0)		
	x2 = np.moveaxis(img_to_array(x2), -1, 0)		
	x3 = np.moveaxis(img_to_array(x3), -1, 0)
	#Create data
	unit.append(x1[0])
	unit.append(x1[1])
	unit.append(x1[2])
	unit.append(x2[0])
	unit.append(x2[1])
	unit.append(x2[2])
	unit.append(x3[0])
	unit.append(x3[1])
	unit.append(x3[2])
	unit=np.asarray(unit)	
	unit = unit.reshape((1, 9, HEIGHT, WIDTH))
	unit = unit.astype('float32')
	unit /= 255
	y_pred = model.predict(unit, batch_size=BATCH_SIZE)
	y_pred = y_pred > 0.5
	y_pred = y_pred.astype('float32')
	h_pred = y_pred[0]*255
	h_pred = h_pred.astype('uint8')
	frame_time = custom_time(cap.get(cv2.CAP_PROP_POS_MSEC))
	if np.amax(h_pred) <= 0:
		f.write(str(count)+',0,0,0,'+frame_time+'\n')
		out.write(image3)
	else:	
		#h_pred
		(cnts, _) = cv2.findContours(h_pred.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
		rects = [cv2.boundingRect(ctr) for ctr in cnts]
		max_area_idx = 0
		max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
		for i in range(len(rects)):
			area = rects[i][2] * rects[i][3]
			if area > max_area:
				max_area_idx = i
				max_area = area
		target = rects[max_area_idx]
		(cx_pred, cy_pred) = (int(ratio*(target[0] + target[2] / 2)), int(ratio*(target[1] + target[3] / 2)))

		f.write(str(count)+',1,'+str(cx_pred)+','+str(cy_pred)+','+frame_time+'\n')
		image3_cp = np.copy(image3)
		cv2.circle(image3_cp, (cx_pred, cy_pred), 5, (0,0,255), -1)
		out.write(image3_cp)
	image1 = image2
	image2 = image3
	success, image3 = cap.read()
	count += 1

f.close()
out.release()
end = time.time()
print('Prediction time:', end-start, 'secs')
print('Done......')

In [ ]:
!python3 '/content/TrackNetv2/3_in_1_out/predict.py'  --save_weights_path='/content/TrackNetv2/3_in_1_out/model_33' --input_video_path='/content/drive/MyDrive/Recording #4.mp4' --output_video_path="/content/drive/MyDrive/tracked.mp4" --n_classes=256